In [8]:
import tensorflow as tf

from corpus import Corpus
import numpy as np

from tools import Timer
from tools import explog
from autoencoders import Autoencoder_ffnn
from genetic import evolution
from genetic import experiment

encode=10
dictsize=1000000
popsize=40


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction=1
corp_path="/mnt/permanent/Language/Hungarian/Corp/Webkorpusz/webkorpusz.wpl"#'/home/velkey/corp/webkorpusz.wpl'
corp=Corpus(corpus_path=corp_path,language="Hun",size=dictsize,encoding_len=10)
all_features=corp.featurize_data_charlevel_onehot(corp.hun_lower)
train=all_features[0:int(len(all_features)*0.8)]
test=all_features[int(len(all_features)*0.8):len(all_features)]
x_train = train.reshape((len(train), np.prod(train.shape[1:])))
x_test = test.reshape((len(test), np.prod(test.shape[1:])))
print(x_train.shape)

logger=explog(encoder_type="Accuracy_tracker_"+str(encode),
              encoding_dim=encode,feature_len=10,
              lang="Hun",unique_words=len(set(corp.full)),
              name="demo"+str(encode),population_size=popsize,
              words=len(corp.hun_lower))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

exp=experiment(encoded_width=10,layermax=10,layermin=2,maxw=10,minw=3,out_dim=360)
exp.weights=[348, 254, 10, 254, 348, 360]
exp.len=len(exp.weights)

Reading file, speed:  2317857.204040096  words/s
Corpus initalized, fields: ['unique', 'lower', 'hun_lower', 'lower_unique', 'hun_lower_unique'] 
Unique words:  135416
(598457, 360)


In [10]:
encoder=Autoencoder_ffnn(experiment=exp,
                 logger=logger,tf_session=sess,
                 inputdim=360,
                 layerlist=exp.weights,
                 encode_index=int(exp.len/2-1),
                 optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                 nonlinear=tf.sigmoid)

In [11]:
encoder.train(x_train,x_test,512,50)

In [38]:
a=x_test[1000:1200,:]
b=(encoder.reconstruct(a))

In [40]:
t=[b[0].reshape(10,36)]
len=100
for i in range(len):
    print(corp.defeaturize_data_charlevel_onehot([a[i].reshape(10,36)])[0],"\t",
          corp.defeaturize_data_charlevel_onehot([b[i].reshape(10,36)])[0])
encoder.char_accuracy(x_test)

   részben 	    részben
 munkaadói 	  haltaanói
        és 	         és
           	 kjslktsülc
           	 kjslktsülc
        is 	         is
      volt 	       volt
         a 	          a
        és 	         és
    jelölt 	     jelélt
  országok 	   országok
           	 kjslktsülc
           	 kjslktsülc
     ebben 	      ebben
         a 	          a
  műfajban 	   korakban
        ez 	         ez
      volt 	       volt
        az 	         az
      első 	       első
rendezvény 	 rendenvény
         a 	          a
 találkozó 	  találhozó
       nem 	        nem
 hivatalos 	  hivatalos
 programja 	   logramja
         a 	          a
       bce 	         de
     által 	      által
 rendezett 	  tervezett
  bőripari 	   kitáhari
           	 kjslktsülc
        és 	         és
   vásáron 	    hiságon
 kezdődött 	  kegködött
szeptember 	 szeptember
  vasárnap 	   kasárnap
      ahol 	       ahol
  várszegi 	   kérszegi
        úr 	         úr
         a 	          a
       bce 	    

0.772544865153895